In [13]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Model, load_model, model_from_json
from keras.layers import Dense, Embedding, Bidirectional, Input, TimeDistributed
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import pandas as pd
from sklearn.metrics import r2_score
import numpy as np
import sys
import h5py
import os
import tensorflow as tf
# tf.enable_eager_execution()

from tensorflow.nn.rnn_cell import LSTMStateTuple
from tensorflow.keras.layers import TimeDistributed
from tensorflow.contrib.rnn import OutputProjectionWrapper

import torch
from torch.autograd import Variable
from torch import optim,nn
from torch.utils.data import TensorDataset, DataLoader
from ignite.metrics import Accuracy, Loss
import pickle

In [14]:
#parameters
maxlen = 30
labels = 2

In [3]:
czech = pd.read_excel('czech.xlsx', encoding='latin',header = None)
czech.columns = ['Name', 'm_or_f']
czech['namelen'] = [len(str(i)) for i in czech['Name']]


In [4]:
cname = czech['Name']
collect = []


for i in range(len(cname)):
    collect.extend(list(str(cname[i]).lower()))
# collect.extend(['END'])
collect = set(collect)

czech.head()

,Name,m_or_f,namelen
0,Abigail,f,7
1,Ada,f,3
2,Adalberta,f,9
3,Adéla,f,5
4,Adelaida,f,8


In [7]:
data_set = pd.read_csv("gender_data.csv",header=None)
data_set.columns = ['name','m_or_f']
data_set['namelen']= [len(str(i)) for i in data_set['name']]
data_set1 = data_set[(data_set['namelen'] >= 2) ]

In [8]:
data_set1.groupby('m_or_f')['name'].count()

m_or_f
f    6705
m    8475
Name: name, dtype: int64

In [9]:
names = data_set['name']
gender = data_set['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
vocab = vocab.union(collect)
len_vocab = len(vocab)

In [10]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of data_set is ",len(data_set1))

{'x', 'b', '6', 'š', 'ě', 'ý', 'o', 'ú', 'z', '0', 'ů', 'm', 'y', 'h', 'l', 'ř', '8', 'ž', 'i', 'p', '3', 'ó', '1', 'é', 'END', 'k', ' ', 'á', '.', 'a', 'c', '2', 'č', '4', 'j', 'ň', 'v', 'g', 'n', 'r', '5', 'í', 'q', 'w', 'u', 'ť', 'ď', 't', 'e', 's', 'd', '7', '9', 'f'}
vocab length is  54
length of data_set is  15226


In [11]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [15]:
# with open('char_index.txt', 'wb') as handle:
#     pickle.dump(char_index, handle)

with open('char_index.txt', 'rb') as handle:
    char_index = pickle.loads(handle.read())


In [16]:
print(char_index)

{'x': 15, '6': 12, 'ě': 24, 'b': 25, 'ú': 26, 'o': 37, 'z': 3, '0': 44, 'm': 28, 'y': 50, 'h': 14, 'l': 0, 'a': 45, 'ř': 31, '8': 53, 'n': 51, 'š': 36, 'ž': 39, 'i': 32, 'p': 5, 'ó': 52, 'ý': 11, '1': 41, 'é': 20, 'END': 43, 'á': 17, ' ': 22, '.': 34, '2': 18, 'c': 42, 'č': 38, '4': 29, 'j': 21, '3': 49, 'ň': 47, 'k': 35, 'v': 19, 'g': 9, 'r': 6, '5': 40, 'í': 4, 'q': 10, 'w': 33, 'ť': 46, 'u': 30, 'ď': 13, '9': 7, 't': 48, 'e': 1, 's': 8, 'd': 16, '7': 2, 'ů': 23, 'f': 27}


In [17]:
#train test split
np.random.seed(0)
msk = np.random.rand(len(data_set1)) < 0.9
train = data_set1[msk]
test = data_set1[~msk]     


In [18]:
def set_flag(i):
    tmp = np.zeros(len_vocab);
    tmp[i] = 1
    return(tmp)

In [19]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

In [20]:
#take data_set upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_x = []
train_y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_x.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_y.append([1, 0])
    else:
        train_y.append([0, 1])
    
train_x=np.asarray(train_x)
train_y=np.asarray(train_y)

In [21]:
test_x = []
test_y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_x.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_y.append([1, 0])
    else:
        test_y.append([0, 1])
    
test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

In [22]:
print(np.asarray(train_x).shape)
print(np.asarray(test_y).shape)

(13713, 30, 54)
(1513, 2)


In [23]:
vtrain_x = []
vtrain_y = []

train_name = [str(i) for i in czech.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtrain_x.append(tmp)
for i in czech.m_or_f:
    if i == 'm':
        vtrain_y.append([1,0])
    else:
        vtrain_y.append([0,1])
vtrain_x = np.asarray(vtrain_x)
vtrain_y = np.asarray(vtrain_y)

### Making BatchLoader

In [24]:
#Creating Tensor Datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

#Now creating Data Loaders
batch = 600

train_loader = DataLoader(train_data, shuffle = True, batch_size = batch)
test_loader = DataLoader(test_data, shuffle = True, batch_size = batch)

dataiter = iter(train_loader)

sample_x, sample_y = dataiter.next()

print("Sample input size of one instance from iter is: ", sample_x.size()) #Batch size, Sequence Length, Feature Dimension
print("Sample target size of that same one instance from iter is", sample_y.size())

Sample input size of one instance from iter is:  torch.Size([600, 30, 54])
Sample target size of that same one instance from iter is torch.Size([600, 2])


#### build model in Tensorflow ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [25]:
''' Parameters '''
tf.reset_default_graph()
no_units=524 

features=train_x.shape[-1] #30
# new_phoneme=new_phoneme.reshape((-1,1,features))

### Making Placeholders

In [26]:
train = tf.placeholder(dtype = tf.float32, shape = [None, None , features], name = 'input')
target = tf.placeholder(dtype = tf.int32, shape = [None, 2], name = 'label')

In [27]:
encoder_fw = [tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]
encoder_bw = [tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]

for i in range(1,3):

    encoder_fw.append(tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]))
    encoder_bw.append(tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]))


In [28]:


encoder_outputs, encoder_fw_state, encoder_bw_state = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                                                                           cells_fw = encoder_fw,
                                                                           cells_bw = encoder_bw,
                                                                           inputs = train,
                                                                           dtype = tf.float32
                                                                          )

In [29]:
# encoder_outputs = tf.concat((encoder_fw_output, encoder_bw_output), 2)

encoder_state_c = tf.concat((encoder_fw_state[-1][0].c, encoder_bw_state[-1][0].c), 1)

encoder_state_h = tf.concat((encoder_fw_state[-1][0].h, encoder_bw_state[-1][0].h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_state_c,
    h=encoder_state_h
)

In [30]:
output=tf.layers.Dense(2)(encoder_state_h)

In [31]:
losss= tf.losses.softmax_cross_entropy(target, output)
trainop =  tf.train.AdamOptimizer(learning_rate=0.001).minimize(losss)



In [32]:
losx=[]
saver = tf.train.Saver()
# with tf.device('/gp):
# with tf.device('/gpu:0'):
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())


    for i in range(15):
        for batch_j, target_j in train_loader:

            loss,_=sess.run([losss,trainop],feed_dict={train: batch_j,
                                                       target : target_j})
            losx.append([loss])
        print("Current epoch going is...", i," and current loss is...",loss)
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)

    



ResourceExhaustedError: OOM when allocating tensor of shape [] and type float
	 [[node stack_bidirectional_rnn/cell_2/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel/Initializer/truncated_normal/stddev (defined at /usr/local/lib/python3.5/dist-packages/tensorflow/contrib/rnn/python/ops/rnn.py:233)  = Const[_class=["loc:@stack...nel/Assign"], dtype=DT_FLOAT, value=Tensor<type: float shape: [] values: 0.0265462045>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'stack_bidirectional_rnn/cell_2/bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel/Initializer/truncated_normal/stddev', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/

In [37]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
with tf.device('/gpu:0'):
    with tf.Session() as sess:
        
#         sess.run(tf.global_variables_initializer())
        if(tf.train.checkpoint_exists("/tmp/model.ckpt")):
            saver.restore(sess, "/tmp/model.ckpt")
        out = sess.run([output], feed_dict={train: np.asarray(X)})
        out = np.squeeze(out)
# pred=model.predict(np.asarray(X))
# print(np.round(out[:,-1,:]))

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


In [38]:
out.shape[0]

9

In [39]:
out = np.argmax(out, axis=1)
for i in range(out.shape[0]):
    if out[i] == 0:
        print(name[i], "....it's female")
    else:
        print(name[i],"....It's male")

sandhya ....It's male
jaspreet ....it's female
rajesh ....it's female
kaveri ....It's male
aditi deepak ....It's male
arihant ....it's female
sasikala ....It's male
aditi ....It's male
ragini rajaram ....It's male


In [ ]:
print("This is",sum([np.argmax(out[:, -1, :][i])==np.argmax(target_j[i].numpy()) for i in range(batch_j.shape[0])])/1000)


In [31]:
#build the model: 2 stacked LSTM
print('Build model...')
input_bilstm=Input(shape = (maxlen,len_vocab))
bi_one = Bidirectional(LSTM(512, return_sequences=True))(input_bilstm)
drop1 = Dropout(0.2)(bi_one)
bi_two = Bidirectional(LSTM(512, return_sequences=False))(drop1)
drop2 = Dropout(0.2)(bi_two)
output = Dense(2, activation='softmax')(drop2)
model = Model(input_bilstm, output)


optimizer = optimizers.adam(lr = 0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_acc',patience=10, verbose=1)
model_checkpoint = ModelCheckpoint('Martin_program.hdf5',monitor='val_acc',save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc',factor=0.5, patience=5, min_lr=0.0001, verbose=1)
print('Model Built')
print("Weights are being imported from previously trained model..")
# model = load_model('Martin_program.hdf5')
print("Weights imported")

Build model...
Model Built
Weights are being imported from previously trained model..
Weights imported


In [23]:
batch_size=500
loaded_model.fit(train_X, train_Y,
          batch_size=batch_size,
          epochs=10,
#           callbacks=[ model_checkpoint,reduce_lr,early_stopping],
          validation_data=(vtrain_x, vtrain_y),
          verbose = 2
         )
# model.save('model.h5')


Train on 13721 samples, validate on 1250 samples
Epoch 1/10
 - 16s - loss: 0.8588 - acc: 0.5744 - val_loss: 0.7034 - val_acc: 0.5312
Epoch 2/10
 - 15s - loss: 0.5976 - acc: 0.6696 - val_loss: 0.6760 - val_acc: 0.5872
Epoch 3/10


KeyboardInterrupt: 

In [39]:
# json_file = open('model_num.json', 'r')

# loaded_model_json = json_file.read()
# json_file.close()

# loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model.hdf5")

# model.save('model_um.hdf5')
# loaded_model = load_model('Martin_program.hdf5')

In [40]:
score, acc = loaded_model.evaluate(vtrain_x, vtrain_y)
print('Test score:', score)
print('Test accuracy:', acc)

# pred = new_model.predict(vtrain_x)
# r2_score(np.around(pred), vtrain_y)

1250/1250 [==============================] - 1s 853us/step
Test score: 1.118419617319107
Test accuracy: 0.4328


In [29]:
# name=[sys.argv[1]]
name = ["Shivam"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=loaded_model.predict(np.asarray(X))
pred = pred.round()[0]
if pred[0] == 1.0:
    print( name[0], ".... is name of a male")
else:
    print(name[0], ".... is name of a female")

Shivam .... is name of a female
